# Base Gas Optimizer - Model Training on Google Colab

This notebook trains all 3 prediction models (1h, 4h, 24h) using the collected data.

**Steps:**
1. Upload `gas_data_colab.db` to Colab (from backend folder)
2. Run all cells
3. Download the trained model files back to your project

In [ ]:
# Install dependencies
!pip install scikit-learn pandas numpy sqlalchemy -q

In [ ]:
import os
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, r2_score
import pickle
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

In [ ]:
# Check if database is uploaded
if not os.path.exists('gas_data_colab.db'):
    print("❌ Please upload gas_data_colab.db file to this Colab session")
    print("   Click the folder icon on the left, then upload the file")
else:
    db_size = os.path.getsize('gas_data_colab.db') / (1024 * 1024)
    print(f"✅ Database found: {db_size:.1f} MB")

In [ ]:
# Load data from database
conn = sqlite3.connect('gas_data_colab.db')

print("📊 Loading gas prices...")
gas_df = pd.read_sql_query("""
    SELECT * FROM gas_prices 
    ORDER BY timestamp
""", conn)
gas_df['timestamp'] = pd.to_datetime(gas_df['timestamp'])

print("📊 Loading onchain features...")
onchain_df = pd.read_sql_query("""
    SELECT * FROM onchain_features 
    ORDER BY timestamp
""", conn)
onchain_df['timestamp'] = pd.to_datetime(onchain_df['timestamp'])

conn.close()

print(f"\n✅ Loaded data:")
print(f"   Gas prices: {len(gas_df):,} records")
print(f"   OnChain features: {len(onchain_df):,} records")
print(f"   Date range: {gas_df['timestamp'].min()} to {gas_df['timestamp'].max()}")

In [ ]:
# Feature engineering function
def add_time_features(df):
    """Add time-based features"""
    df = df.copy()
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    return df

def add_lag_features(df, column, lags):
    """Add lag features for a column"""
    df = df.copy()
    for lag in lags:
        df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    return df

def add_rolling_features(df, column, windows):
    """Add rolling statistics"""
    df = df.copy()
    for window in windows:
        df[f'{column}_rolling_mean_{window}'] = df[column].rolling(window=window, min_periods=1).mean()
        df[f'{column}_rolling_std_{window}'] = df[column].rolling(window=window, min_periods=1).std().fillna(0)
    return df

print("✅ Feature engineering functions defined")

In [ ]:
# Prepare training data
print("📊 Preparing training data...\n")

# Merge gas prices with onchain features
df = pd.merge_asof(
    gas_df.sort_values('timestamp'),
    onchain_df.sort_values('timestamp'),
    on='timestamp',
    direction='nearest',
    tolerance=pd.Timedelta('5min')
)

print(f"✅ Merged data: {len(df):,} samples")

# Add time features
df = add_time_features(df)

# Add lag features for gas price (1, 2, 3, 6, 12 periods back)
df = add_lag_features(df, 'gas_price', [1, 2, 3, 6, 12])

# Add rolling statistics (5, 15, 60 minute windows)
df = add_rolling_features(df, 'gas_price', [5, 15, 60])

# Handle enhanced features if present
enhanced_features = ['pending_tx_count', 'unique_addresses', 'tx_per_second', 'gas_utilization_ratio']
for feat in enhanced_features:
    if feat in df.columns:
        df[feat] = df[feat].fillna(df[feat].median())

# Add derived congestion features
if 'pending_tx_count' in df.columns and 'tx_per_second' in df.columns:
    df['congestion_level'] = df['pending_tx_count'] * df['tx_per_second']
    df['is_highly_congested'] = ((df['pending_tx_count'] > df['pending_tx_count'].quantile(0.75)) & 
                                  (df['tx_per_second'] > df['tx_per_second'].quantile(0.75))).astype(int)

# Drop rows with NaN (from lag features)
df = df.dropna()

print(f"✅ Prepared {len(df):,} samples with {len(df.columns)} features")
print(f"\n📊 Feature columns: {df.columns.tolist()[:10]}...")

In [ ]:
# Create prediction targets
def create_targets(df, horizons):
    """Create target variables for different prediction horizons"""
    df = df.copy()
    for horizon_name, minutes in horizons.items():
        periods = minutes  # Since we're sampling every minute
        df[f'target_{horizon_name}'] = df['gas_price'].shift(-periods)
    return df

horizons = {'1h': 60, '4h': 240, '24h': 1440}
df = create_targets(df, horizons)
df = df.dropna()  # Remove rows where we can't create targets

print(f"✅ Created prediction targets for {list(horizons.keys())}")
print(f"   Final training samples: {len(df):,}")

In [ ]:
# Define feature columns
feature_cols = [col for col in df.columns if col not in 
                ['timestamp', 'id', 'gas_price', 'target_1h', 'target_4h', 'target_24h']]

print(f"📊 Using {len(feature_cols)} features for training:")
print(f"   {feature_cols}")

In [ ]:
# Training function
def train_model(X, y, horizon_name):
    """Train a single model with time-series cross-validation"""
    print(f"\n{'='*60}")
    print(f"🎯 Training model for {horizon_name} prediction horizon")
    print(f"{'='*60}")
    print(f"   Samples: {len(X):,}")
    print(f"   Features: {X.shape[1]}")
    
    # Scale features using RobustScaler (handles outliers better)
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)
    print("✅ Features scaled with RobustScaler")
    
    # Time-series cross-validation
    print("📊 Running time-series cross-validation...")
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_scaled), 1):
        X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # Train RandomForest
        model = RandomForestRegressor(
            n_estimators=100,
            max_depth=15,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X_train, y_train)
        
        # Validate
        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        
        # Directional accuracy
        direction_correct = np.sum((y_pred > y_train.iloc[-1]) == (y_val.values > y_train.iloc[-1]))
        dir_acc = direction_correct / len(y_val)
        
        cv_scores.append({'mae': mae, 'r2': r2, 'dir_acc': dir_acc})
        print(f"   Fold {fold}: MAE={mae:.4f}, R²={r2:.4f}, Dir Acc={dir_acc:.2%}")
    
    # Print average scores
    avg_mae = np.mean([s['mae'] for s in cv_scores])
    avg_r2 = np.mean([s['r2'] for s in cv_scores])
    avg_dir = np.mean([s['dir_acc'] for s in cv_scores])
    print(f"\n📊 Cross-validation results:")
    print(f"   Average MAE: {avg_mae:.4f}")
    print(f"   Average R²: {avg_r2:.4f}")
    print(f"   Average Directional Accuracy: {avg_dir:.2%}")
    
    # Train final model on all data
    print(f"\n🎯 Training final model on full dataset...")
    final_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42,
        n_jobs=-1
    )
    final_model.fit(X_scaled, y)
    print("✅ Final model trained")
    
    return {
        'model': final_model,
        'scaler': scaler,
        'feature_names': feature_cols,
        'metrics': {
            'mae': avg_mae,
            'r2': avg_r2,
            'directional_accuracy': avg_dir
        }
    }

print("✅ Training function defined")

In [ ]:
# Train all models
print("\n" + "="*70)
print("🚀 STARTING MODEL TRAINING")
print("="*70)

models = {}

for horizon_name in ['1h', '4h', '24h']:
    X = df[feature_cols]
    y = df[f'target_{horizon_name}']
    
    trained = train_model(X, y, horizon_name)
    models[horizon_name] = trained
    
print("\n" + "="*70)
print("✅ ALL MODELS TRAINED SUCCESSFULLY")
print("="*70)

In [ ]:
# Print summary
print("\n📊 TRAINING SUMMARY")
print("="*70)
for horizon_name, model_data in models.items():
    metrics = model_data['metrics']
    print(f"\n{horizon_name} Model:")
    print(f"   MAE: {metrics['mae']:.4f} Gwei")
    print(f"   R² Score: {metrics['r2']:.4f}")
    print(f"   Directional Accuracy: {metrics['directional_accuracy']:.2%}")

In [ ]:
# Save models
print("\n💾 Saving models...")

os.makedirs('saved_models', exist_ok=True)

for horizon_name, model_data in models.items():
    filename = f'saved_models/model_{horizon_name}.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model_data, f)
    file_size = os.path.getsize(filename) / 1024
    print(f"✅ Saved {filename} ({file_size:.1f} KB)")

print("\n" + "="*70)
print("🎉 TRAINING COMPLETE!")
print("="*70)
print("\n📦 Next steps:")
print("   1. Download the 'saved_models' folder from Colab")
print("   2. Replace the models in your backend/models/saved_models/ directory")
print("   3. Restart your backend server to use the new models")

In [ ]:
# Optional: Create a zip file for easy download
!zip -r models.zip saved_models/
print("\n✅ Created models.zip - download this file from the Files panel")